In [3]:
import cv2

img = cv2.imread('../Data/elephant.jpg')
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


cv2.imshow('Input Image', img)
cv2.waitKey(0)

cell_size = (8, 8) 

block_size = (2, 2) 

nbins = 9


hog = cv2.HOGDescriptor(_winSize=(gray.shape[1] // cell_size[1] * cell_size[1],
                                  gray.shape[0] // cell_size[0] * cell_size[0]),
                        _blockSize=(block_size[1] * cell_size[1],
                                    block_size[0] * cell_size[0]),
                        _blockStride=(cell_size[1], cell_size[0]),
                        _cellSize=(cell_size[1], cell_size[0]),
                        _nbins=nbins)

n_cells = (gray.shape[0] // cell_size[0], gray.shape[1] // cell_size[1])


hog_feats = hog.compute(gray).reshape(n_cells[1] - block_size[1] + 1,
                        n_cells[0] - block_size[0] + 1,
                        block_size[0], block_size[1], nbins).transpose((1, 0, 2, 3, 4))



cv2.destroyAllWindows()